In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

from lib.data.datareader_total import DataReaderTotal
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_fit3d import DataReaderFIT3D
from lib.utils.args import get_opts_args, get_opt_args_from_model_name
from lib.model.load_model import load_model
from lib.model.load_dataset import load_dataset
from lib.data.dataset_motion_3d import MotionDataset3DTotal
from lib.model.evaluation import *

### Evaluate single experiment

In [2]:
checkpoint = 'MB_train_h36m_gt'
args, opts = get_opt_args_from_model_name(checkpoint)
model_pos, chk_filename, checkpoint = load_model(opts, args)

subset = 'FIT3D-GT-ALL_TEST'
args.subset_list = [subset]
train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)

# if '3DHP' in subset:
#     args.eval_part = 'whole_without_nose'
#     args.part_list = ['whole_without_nose']

configs/pose3d/MB_train_h36m_gt.yaml
True
MB_train_h36m_gt
Loading checkpoint checkpoint/pose3d/MB_train_h36m_gt/best_epoch.bin


In [6]:
e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)

INFO: Testing
No epoch information in the checkpoint


  0%|          | 0/411 [00:00<?, ?it/s]

100%|██████████| 411/411 [04:14<00:00,  1.61it/s]


No eval_keypoint. Use part list
Part: whole
+------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-----------------------+------------------------+-----------------------+---------------------+----------------------------------+------------------------+--------------------+--------------------+--------------------+---------------------------------+-------------------+-----------------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+---------------

In [17]:
save_folder = f'/home/hrai/codes/MotionBERT/saved_results/{args.model}'
save_path = os.path.join(save_folder, f'{subset}.pkl')
print(save_path)
results_dict = {'results_all': results_all, 'gts_all': gts_all, 'inputs_all': inputs_all, 'total_result_dict': total_result_dict}
savepkl(results_dict, save_path)

/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt/FIT3D-GT-ALL_TEST.pkl


### Evaluate and save all experiments

In [2]:
blacklist_checkpoint = ['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_s15678_tr_54138969_ts_others',
                        'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_dist',
                        'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_tr_s1_ts_s5678',
                        'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_dist_input_centering_tr_s1_ts_s5678',
                        'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_dist_s15678_tr_54138969_ts_others',
                        'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_dist_tr_s1_ts_s5678',
                        'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5',
                        ]

In [3]:
experiment_list = [item.split('.')[0] for item in os.listdir('experiments')]
result_list = os.listdir('saved_results')
whitelist_checkpoint = []
for item in os.listdir('configs/pose3d'):
    checkpoint = item.split('.')[0]
    if 'cam_no_factor' not in checkpoint: continue
    if checkpoint in blacklist_checkpoint: continue
    if checkpoint not in experiment_list:
        whitelist_checkpoint.append(checkpoint)
    else:
        subset_list = []
        with open(f'experiments/{checkpoint}.txt', 'r') as f:
            lines = f.readlines()
            for line in lines:
                subset = line.split('\n')[0]
                subset_list.append(subset)
        for subset in subset_list:
            if not os.path.exists(f'saved_results/{checkpoint}/{subset}.pkl'):
                print(f'{checkpoint}/{subset} not found')
                whitelist_checkpoint.append(checkpoint)

In [4]:
whitelist_checkpoint

[]

In [5]:
experiment_root = f'/home/{user}/codes/MotionBERT/experiments'
for item in glob(experiment_root+'/*.txt'):
    checkpoint = os.path.basename(item).split('.')[0]
    if checkpoint in blacklist_checkpoint: continue
    if len(whitelist_checkpoint) > 0 and checkpoint not in whitelist_checkpoint: continue
    if 'cam_no_factor' not in checkpoint: continue
    save_folder = f'/home/hrai/codes/MotionBERT/saved_results/{checkpoint}'
    if not os.path.exists(save_folder): os.makedirs(save_folder)
    
    # load model
    args, opts = get_opt_args_from_model_name(checkpoint, verbose=False)
    model_pos, chk_filename, checkpoint = load_model(opts, args)
    
    subset_list = []
    with open(item, 'r') as f:
        lines = f.readlines()
        for line in lines:
            subset = line.strip()
            subset_list.append(subset)
    
    for subset in subset_list:
        save_path = os.path.join(save_folder, f'{subset}.pkl')
        if os.path.exists(save_path): 
            print(f'{save_path} exists')
            continue
        try:
            # load dataset
            args.subset_list = [subset]
            train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)
            # evaluation
            e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)
            # save results
            results_dict = {'results_all': results_all, 'gts_all': gts_all, 'inputs_all': inputs_all, 'total_result_dict': total_result_dict, 'e1': e1, 'e2': e2}
            savepkl(results_dict, save_path)
        except Exception as e:
            print(f'Error: {e}')


MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1_ts_s5678
Loading checkpoint checkpoint/pose3d/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1_ts_s5678/best_epoch.bin
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1_ts_s5678/3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5_ADAPTIVE_FOCAL-TEST_ALL_TRAIN.pkl exists
/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1_ts_s5678/3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5_ADAPTIVE_FOCAL-TEST_TS1_6.pkl exists
Loading dataset...
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5_ADAPTIVE_FOCAL-ALL_TEST
[overwrite: False] ==> Loading FIT3D source_list...
[overwrite: False] ==> Loading FIT3D cam_param_adaptive_f

100%|██████████| 1504/1504 [01:03<00:00, 23.65it/s]


[overwrite: False] ==> Loading FIT3D cam_3d_fixed_dist_5...
[overwrite: False] ==> Loading FIT3D img_2d_canonical_fixed_dist_5_adaptive_focal...


100%|██████████| 1504/1504 [00:00<00:00, 1992.02it/s]


[overwrite: False] ==> Loading FIT3D cam_3d_canonical_fixed_dist_5...
INFO: Testing
No epoch information in the checkpoint


...

No eval_keypoint. Use part list
Part: whole
+------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-----------------------+------------------------+-----------------------+---------------------+----------------------------------+------------------------+--------------------+-------------------+--------------------+---------------------------------+-------------------+-----------------------------+----------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------

100%|██████████| 411/411 [02:16<00:00,  3.02it/s]


No eval_keypoint. Use part list
Part: whole
+------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-----------------------+------------------------+-----------------------+---------------------+----------------------------------+------------------------+--------------------+-------------------+--------------------+---------------------------------+-------------------+-----------------------------+----------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------

100%|██████████| 411/411 [02:16<00:00,  3.02it/s]


No eval_keypoint. Use part list
Part: whole
+------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-----------------------+------------------------+-----------------------+---------------------+----------------------------------+------------------------+--------------------+--------------------+--------------------+---------------------------------+--------------------+-----------------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------

### Cherrypick e1, e1 -> result_dict.pkl

In [2]:
blacklist_checkpoint = ['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_s15678_tr_54138969_ts_others']


if os.path.exists('result_dict.pkl'):
    print('Loading result_dict.pkl')
    result_dict = readpkl('result_dict.pkl')
else:
    result_dict = {}
result_root = '/home/hrai/codes/MotionBERT/saved_results'
checkpoint_list = os.listdir(result_root)
for checkpoint in tqdm(checkpoint_list):
    if checkpoint in blacklist_checkpoint: continue # skip
    if checkpoint not in result_dict.keys(): result_dict[checkpoint] = {} # create new key if not exists
    checkpoint_root = os.path.join(result_root, checkpoint)
    subset_list = glob(checkpoint_root+'/*.pkl')
    for item in subset_list:
        subset = os.path.basename(item).split('.')[0]
        if subset in result_dict[checkpoint].keys(): 
            #print(f'{subset} exists')
            continue
        result = readpkl(item)
        result_dict[checkpoint][subset] = {'e1': result['e1'], 'e2': result['e2']}

Loading result_dict.pkl


100%|██████████| 29/29 [00:00<00:00, 27814.96it/s]


In [3]:
savepkl(result_dict, 'result_dict.pkl')